# Assignment

We will do a link prediction challenge. You will be provided with a network to train your prediction algorithm. You are also provided with a table that presents pairs of nodes, with which we evaluate the link prediction performance of your algorithm. Your task will be to calculate the probability of edges for each node pair in the table and submit both the probability of edges and your code.

## Data

The train and test edge sets are located under "dataset" directory.

In [ ]:
import pandas as pd

edge_table = pd.read_csv("https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/link-prediction-dataset/train-edges.csv")
eval_edge_table = pd.read_csv("https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/link-prediction-dataset/test-edges.csv")

The edge_table is a collection of the observed edges within the network, with which you can train your link prediction algorithm. On the other hand, the eval_edge_table consists of pairs of nodes that will be used to assess the algorithm's performance. This table includes both connected and unconnected node pairs, and the information about whether two nodes are connected or not is concealed from you.

## Evaluation

The performance of an algorithm is evaluated based on [ROC-AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic).

## How to develop a link prediction algorithm

### Computing the structural features
Link prediction is essentially a binary classification task, where you are given a set of node pairs, and your task is to *classify* them into connected and unconnected node pairs. Your classification must be based on *features* of the node pairs that characterize the structure around the node pair. For instance, one can consider degrees of nodes $i$ and $j$, and/or the number of common neighbors, as a feature of the node pair.

![](https://pythonnumericalmethods.berkeley.edu/_images/25.01.03-representation_of_data.png)

More formally, one will encode the structural features of node pairs as a single matrix $X$, where each row describes the feature of a node pair. You also have a target vector $y$, which is a value you want to output based on the feature vector. In link prediction, our target is absence/presence of edges, so $y$ is vector of binary variables. The link predictor is a function $f$ that predicts $y$ from $X$, i.e.,
$$
\hat y := f(X)
$$
and we want to train $f$ such that its output aligns with the target $y$, i.e., $y\sim \hat y$.

### Splitting data into train and test data

There are countless ways to design predictor $f$. One option is to design the function yourself, while another is to utilize machine learning to train a predictive function $f$. Regardless of the approach, it is crucial to have *test data* to assess the accuracy and effectiveness of your prediction function $f$.

The test data should be independent of the training data. In link prediction task, data are a set of edges, so we put aside some edges for testing (test edge set), and train the link predictor with the remaining edges (i.e., training edge set).

In [ ]:
from sklearn.model_selection import train_test_split

train_edge_table, test_edge_table = train_test_split(edge_table, test_size=0.2)

This test_edge_table consists only of the connected nodes and thus incomplete, since we need unconnected node pairs to assess the performance. So we sample unconnected node pairs and add them to the test_edge_table. Sampling unconnected node pairs is tedious task, so let's use an existing library to do it.

In [ ]:
# Installing a package for link prediction
# https://github.com/skojaku/linkpred/tree/main
!pip install git+https://github.com/skojaku/linkpred.git

In [ ]:
# Create the train network
import numpy as np
from scipy import sparse
import igraph

g = igraph.Graph.DataFrame(
    train_edge_table[["src", "trg"]],
    directed=False,
)
train_net = g.get_adjacency_sparse()

In [ ]:
# Sample unconnected node pairs
from linkpred.LinkPredictionDataset import NegativeEdgeSampler

sampler = NegativeEdgeSampler(negative_edge_sampler="uniform")
sampler.fit(train_net)
neg_src, neg_trg = sampler.sampling(size=test_edge_table.shape[0])

# Add the negative edges to the test edge table, with a new column "isConnected" indicating the presence (=1) or absence (=0) of an edge between the two nodes in the row.
neg_test_edge_table = pd.DataFrame(
    {"src": neg_src, "trg": neg_trg, "isConnected": np.zeros_like(neg_trg)}
)
test_edge_table["isConnected"] = 1
test_edge_table = pd.concat([neg_test_edge_table, test_edge_table])

### Example: a naive predictor based on node degree

Let me demonstrate a naive link prediction algorithm based on degree.
- *Feature variables*: For a node pair $(i,j)$, I compose a feature $[d_i, d_j]$, where $d_i$ is the degree of node $i$ and create a feature matrix.
- *Link predictor $f$*: I will use [a logistic regression model](https://en.wikipedia.org/wiki/Logistic_regression) to compute the probability of edges.

First, I define a function to compute the feature vectors:

In [ ]:
def compute_feature_vectors(pred_src, pred_trg, A):
    deg = np.array(A.sum(axis=1)).reshape(-1)
    X = np.vstack([deg[pred_src], deg[pred_trg]]).T
    return X

Second, I define my link prediction model

In [ ]:
from sklearn.linear_model import LogisticRegression

predictor = LogisticRegression(random_state=0)

Third, I generate the training data ($X$ and $y$) with train_net

In [ ]:
# Get the connected node pairs
pos_src, pos_trg, _ = sparse.find(train_net)

# Get the same number of unconnected node pairs
neg_src, neg_trg = sampler.sampling(size=len(pos_src))

# Merge the connected and unconnected node pairs
src_train, trg_train = np.concatenate([pos_src, neg_src]), np.concatenate(
    [pos_trg, neg_trg]
)

# Generate X and y for training
X_train = compute_feature_vectors(src_train, trg_train, train_net)
y_train = np.concatenate([np.ones_like(pos_src), np.zeros_like(neg_trg)])

And train my model

In [ ]:
predictor.fit(X_train, y_train)

Now, I have the trained model and want to evaluate the performance of the prediction. I employ the AUC-ROC score, with a higher value indicating a higher predictability of links.

In [ ]:
# Compute the features for the test edges
src_test, trg_test, y_test = tuple(
    test_edge_table[["src", "trg", "isConnected"]].values.T
)
X_test = compute_feature_vectors(src_test, trg_test, train_net)

# Prediction
ypred = predictor.predict(X_test)

from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_test, ypred)
print(score)

The highest value of AUC-ROC is 1, and random prediction (predicting presence and absence randomly with probability proportional to the class sizes) corresponds to AUC-ROC value of 0.5.

Now, let's include additional feature, the product of degree. Would the prediction improve?

In [ ]:
def compute_feature_vectors(pred_src, pred_trg, A):
    deg = np.array(A.sum(axis=1)).reshape(-1)
    X = np.vstack([deg[pred_src], deg[pred_trg], deg[pred_src] * deg[pred_trg]]).T
    return X


X_train = compute_feature_vectors(src_train, trg_train, train_net)
predictor = LogisticRegression(random_state=0)
predictor.fit(X_train, y_train)

X_test = compute_feature_vectors(src_test, trg_test, train_net)
ypred = predictor.predict(X_test)

score = roc_auc_score(y_test, ypred)
print(score)

Improved? You can add any structural features by redefining the compute_feature_vectors function. You can also use different machine learning models such as [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html), [Random forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [XGBDboost](https://xgboost.readthedocs.io/en/stable/).

# Tips
1. It is tempting to pack as many features as possible into a feature vector. However, adding more features does not always improve the prediction due to overfitting.
2. Gaining a deep understanding of the data establishes a solid foundation upon which applications can be built. One effective approach is to visualize the network, identify patterns, quantify them, and utilize them as structural features for prediction. This process often yields valuable insights.
3. When it comes to crafting features, you have the option to either create your own or utilize established structural variables that are known to be beneficial for prediction. To delve further into this topic, I recommend exploring the following papers:

#### References
- [Friends and neighbors on the Web - ScienceDirect](https://www.sciencedirect.com/science/article/abs/pii/S0378873303000091?via%3Dihub)
- [Phys. Rev. E 76, 046115 (2007) - Bipartite network projection and personal recommendation](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.76.046115)
[Predicting missing links via local information | SpringerLink](https://link.springer.com/article/10.1140/epjb/e2009-00335-8)
-  [A new status index derived from sociometric analysis | SpringerLink](https://link.springer.com/article/10.1007/BF02289026)
- [The link prediction problem for social networks | Proceedings of the twelfth international conference on Information and knowledge management](https://dl.acm.org/doi/abs/10.1145/956863.956972?casa_token=aV4JEyGvR9sAAAAA:AaLUWuE0wqD3Uo0Fk7uxPdr_qzswAlo93UhtqoXZst75WqouUDVpUhNsGA9i7fvqXR3dNvWNqJk)
- [The anatomy of a large-scale hypertextual Web search engine - ScienceDirect](https://www.sciencedirect.com/science/article/abs/pii/S016975529800110X)
- [SimRank | Proceedings of the eighth ACM SIGKDD international conference on Knowledge discovery and data mining](https://dl.acm.org/doi/abs/10.1145/775047.775126?casa_token=2dAvlEOGymQAAAAA:aqRndkwIV1RSqb5TE2cCzX2A2ZWtdR129VPhO7lRIdjxPGy5BcBXklphX031mtXyFh18_u-LljA)
- [Phys. Rev. E 80, 046122 (2009) - Similarity index based on local paths for link prediction of complex networks](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.80.046122)
- [Link prediction based on local random walk - IOPscience](https://iopscience.iop.org/article/10.1209/0295-5075/89/58007/meta?casa_token=z6kfr7NvfJwAAAAA:xl16SbXlF4oq7-HcR98piZjP5eu4Smo3glqYx82QyW-W1NryrPqsXiZjCEGBy2ZO3cQH79BzswEBufOpIA)
-[Learning spectral graph transformations for link prediction | Proceedings of the 26th Annual International Conference on Machine Learning](https://dl.acm.org/doi/abs/10.1145/1553374.1553447?casa_token=Mm_GiduXOhcAAAAA:HrkFEiCEs6F-graJZzsbxkE0rRtVprEhM-Ro8UYmBKG-rzGo4eXEPDmmomJHMlHpDdsx6-vWdWo)
- [Stacking models for nearly optimal link prediction in complex networks | PNAS](https://www.pnas.org/doi/10.1073/pnas.1914950117)

# Submit your results

0. Design your link prediction algorithm.
1. Load the test edge list from `https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/link-prediction-dataset/test-edges.csv`
2. Compute the likelihood of edges for each node pair in the table
3. Append a column "score" that contains your prediction score
4. Save it to a csv file using pandas.to_csv API and name the file as [student_id].csv

When you prepared your table, run the following command to check if your format is in line with the specification above.

In [ ]:
# Load the edges
df = pd.read_csv(your_file_name)

# The csv file has the columns specified
assert "src" in df.columns
assert "trg" in df.columns
assert "score" in df.columns

# The AUC-ROC score can be calulated
from sklearn.metrics import roc_auc_score

score = roc_auc_score(np.random.randint(0, 1, size=df.shape[0]), df["score"].values)

# The score has no NaN values
assert not np.any(pd.isna(score))

If you pass the test, submit the result to xxxx #todo and your code to #xxx in Jupyter notebook format, with descriptions about your algorithm.